In [ ]:
import os

# Подивимось на структуру папок датасету
data_path = "/kaggle/input"

for dirname, _, filenames in os.walk(data_path):
    # Показуємо тільки назви папок (класи), не всі файли
    level = dirname.replace(data_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(dirname)}/')
    if level >= 2:  # не заглиблюємось далі 2 рівнів
        continue

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

In [ ]:
# Трансформації для тренувальних даних (з аугментацією)
# Посилання: https://pytorch.org/vision/stable/transforms.html
# Навіщо це потрібно: нейромережа вчиться краще, коли бачить більше різноманітних прикладів.
# Без аугментації вона може "завчити" конкретні фото замість того,
# щоб зрозуміти загальні ознаки фрукта.
# Аугментація — це коли ти штучно "розмножуєш" свої фотографії, трохи змінюючи кожну.
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),           # Змінюємо розмір до 224x224
    transforms.RandomHorizontalFlip(p=0.5),  # Випадковий горизонтальний переворот
    transforms.RandomRotation(10),           # Випадковий поворот до 10 градусів
    transforms.ToTensor(),                   # Конвертуємо в тензор [0, 1]
])

# Трансформації для тестових даних (без аугментації)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),           # Тільки змінюємо розмір
    transforms.ToTensor(),                   # Конвертуємо в тензор
])

In [ ]:
# Шлях до даних 
# Cтруктура у Блоку 1
train_path = "/kaggle/input/datasets/sshikamaru/fruit-recognition/train/train"
test_path = "/kaggle/input/datasets/sshikamaru/fruit-recognition/test"

# Створюємо датасети з відповідними трансформаціями
train_dataset = datasets.ImageFolder(train_path, transform=train_transform)
test_dataset = datasets.ImageFolder(test_path, transform=test_transform)

print(f"Кількість тренувальних зразків: {len(train_dataset)}")
print(f"Кількість тестових зразків: {len(test_dataset)}")

In [ ]:
# Назви класів (фруктів)
print(f"Кількість класів: {len(train_dataset.classes)}")
print(f"Класи: {train_dataset.classes}")
print(f"\nclass_to_idx: {train_dataset.class_to_idx}")

In [ ]:
# Подивимось на перший зразок
img, label = train_dataset[0]
print(f"Тип: {type(img)}")
print(f"Форма тензора: {img.shape}")  # [C, H, W] = [3, 224, 224]
print(f"Мітка (індекс): {label}")
print(f"Назва класу: {train_dataset.classes[label]}")

# Перший зразок з кожного класу (до 5 класів)
shown_classes = set()
for i in range(len(train_dataset)):
    img, label = train_dataset[i]
    if label not in shown_classes:
        print(f"Зразок [{i}]: форма {img.shape}, мітка: {label} → {train_dataset.classes[label]}")
        shown_classes.add(label)
    if len(shown_classes) >= 5:
        break

In [ ]:
# Розмір батчу
batch_size = 32

# Створюємо DataLoader для тренувальних та тестових даних
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Перевірка
for images, labels in train_loader:
    print(f"Форма батчу зображень: {images.shape}")  # [batch_size, 3, 224, 224]
    print(f"Форма батчу міток: {labels.shape}")       # [batch_size]
    break  # Виводимо тільки перший батч

In [ ]:
# Візуалізація кількох зразків з тренувального набору
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i, ax in enumerate(axes.flat):
    img, label = train_dataset[i]
    # Конвертуємо тензор [C, H, W] → [H, W, C] для matplotlib
    img_np = img.permute(1, 2, 0).numpy()
    ax.imshow(img_np)
    ax.set_title(train_dataset.classes[label])
    ax.axis('off')

plt.suptitle("Зразки з тренувального датасету", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Отримуємо один батч
images, labels = next(iter(train_loader))

# Створюємо сітку зображень
grid = make_grid(images[:16], nrow=4, padding=2)

# Відображаємо
plt.figure(figsize=(12, 12))
plt.imshow(grid.permute(1, 2, 0).numpy())
plt.title("Батч зображень з тренувального DataLoader")
plt.axis('off')
plt.show()

# Виводимо мітки
print("Мітки:", [train_dataset.classes[l] for l in labels[:16].tolist()])